To start, press the Run All button at the top of this notebook to see your results.

I start by importing pandas.

In [15]:
import pandas as pd

Next I read in my first dataset and asign it to a DataFrame, then I rename the "Duration (min)" column to "Runtime", once that is done I filter the columns I want to use into a new DataFrame and remove all the NaN values.

In [16]:
data = pd.read_csv("imdb_data\imdb-movies-dataset.csv")

data = data.rename(columns={"Duration (min)": "Runtime"})

new_data = data.loc[:, ['Title', 'Runtime', 'Genre', 'Rating']]
new_data = new_data.dropna()

After I removed the NaN values, I needed to convert the "Runtime" column to a string in order to replace the trailing '.0' with an empty space ''.

In [17]:
new_data['Runtime'] = new_data['Runtime'].astype(str)
new_data['Runtime'] = new_data['Runtime'].str.replace('.0', '')

Now that my first dataset is ready, I moved on to my second dataset. I start with reading the CSV into a DataFrame and renaming the columns I want to use so they match my first dataset, then I filter out the columns I want to use into a new DataFrame.

In [18]:
data1k = pd.read_csv("imdb_top1k\imdb_clean.csv")

data1k = data1k.rename(columns={"title": "Title", "runtime": "Runtime", "genre": "Genre", "rating": "Rating"})

new_data1k = data1k.loc[:, ["Title", "Runtime", "Genre", "Rating"]]

My new DataFrame had an issue with having multiple rows for the same film because the films fell into more than one genre. So to combine all the mostly duplicate rows, I grouped all the columns except the 'Genre' column and then using apply to use the lambda function to join all the seperate genre rows into a single string separated by a comma and space, once that was done I had to reset the index. 

In [19]:
new_data1k_grouped = new_data1k.groupby(['Title', 'Runtime', 'Rating'])['Genre'].apply(lambda x: ', '.join(x)).reset_index()

After I had combined the duplicate rows I had to convert the 'Runtime' column on my second dataset to a string so I could use concat to merge my two datasets into my new one, I also had to use ignore_index to reset the index to a new one instead of using the indicies from the original two datasets.

In [20]:
new_data1k_grouped['Runtime'] = new_data1k_grouped['Runtime'].astype(str)

all_data = pd.concat([new_data, new_data1k_grouped], ignore_index=True)

Once all my data was in a single DataFrame, I wrote the data to a new CSV that is stored in my project folder.

In [21]:
all_data.to_csv("all_imdb_data.csv")

Next we will define our parameters for the genre_extractor function by prompting the user to input the genre and lowest rating they want to see ex:(Comedy, 7.5) and then convert the rating to an integer.

In [22]:
genre_str = input("Enter genre")
rating_int = int(input("Enter the lowest rating you want to see"))

Using a function to make a copy of the DataFrame so the original dataset is not altered, we create a filtered DataFrame based on the genre and rating range input by the user and read it to its own CSV.


*Tip: If you want to run program mutiple times to create multiple DataFrames for each genre, make sure to edit the name of the CSV file the filtered_df is being written to anywhere it appears (ex: run program once to filtered_data.csv, then change CSV name to filtered_data1.csv and run it again on a different genre.)

In [23]:
def genre_extractor(df, genre_str, rating_int):
 
    df_copy = df.copy()
    
   
    filtered_df = df_copy[(df_copy['Genre'] == genre_str) & (df_copy['Rating'] >= rating_int)]
    
   
    filtered_df.to_csv('filtered_data.csv', index=False)
    
    return filtered_df


Now we will call the genre_extractor function.

In [24]:
genre_extractor(all_data, genre_str, rating_int)

,Title,Runtime,Genre,Rating
64,The Shawshank Redemption,142,Drama,9.3
145,Fight Club,139,Drama,8.8
184,American Beauty,122,Drama,8.3
238,Requiem for a Dream,102,Drama,8.3
379,The Help,146,Drama,8.1
...,...,...,...,...
10530,Udaan,134,Drama,8.1
10531,Umberto D.,89,Drama,8.2
10560,Who's Afraid of Virginia Woolf?,131,Drama,8.0
10566,Winter Light,81,Drama,8.0


The next function will take the DataFrame created based on your genre and rating inputs and calculate the average runtime.

*Tip: If you are running program multiple times, make sure to edit the CSV file being read in.

In [25]:
def avg_runtime(genre_extractor):
    
    filtered_df = pd.read_csv('filtered_data.csv')

    avg_runtime = filtered_df.Runtime.mean().round() 

    return avg_runtime

Then you will call the 'avg_runtime' function to see what the Golden Movie Ratio is for the genre you selected.

In [26]:
avg_runtime(genre_extractor)

133.0